# Descripcion del proyecto

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.

Crea un modelo con el máximo valor F1 posible. Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba. 

Además, debes medir la métrica AUC-ROC y compararla con el valor F1.

# Tabla de contenido
1. [Inicializacion](#Inicializacion)
2. [Informacion general](#Informacion-general)
3. [Segmenta los datos en entrenamiento, validacion y prueba](#Segmetar-los-datos-en-entrenamiento-validacion-y-prueba)

   4.1.[Arbol de decision](#Arbol-de-decision)
   
   4.2.[Bosque aleatorio](#Bosque-aleatorio)
   
   4.3.[Regresion logistica](#Regresion-logistica)
   
   
5. [Mejora la calidad del modelo](#Mejora-la-calidad-del-modelo)

    
   5.1.[Aplicacion del sobremuestreo para los modelos](#Aplicacion-del-sobremuestreo-para-los-modelos)
    
    
   5.2.[Arbol de decision](#Arbol-de-decision)
   
   
   5.3.[Bosque aleatorio](#Bosque-aleatorio)
   
   
   5.4.[Regresion logistica](#Regresion-logistica)
   
    
6. [Aplicacion del submuestro para los modelos](#Aplicacion-del-submuestreo-paralos-modelos)
    
    6.1.[Arbol de decision](#Arbol-de-decision)
   
   6.2.[Bosque aleatorio](#Bosque-aleatorio)
   
   6.3.[Regresion logistica](#Regresion-logistica)
       
7. [Realiza la prueba final](#Realiza-la-prueba-final)

    
   7.1.[Se realizara la prueba final con el sobremuestreo con los modelos](#Se-realizara-la-prueba-final-con-el-sobremuestreo-con-los-modelos)
   
   
   7.2.[Arbol de decision](#Arbol-de-decision)
   
   7.3.[Bosque aleatorio](#Bosque-aleatorio)
   
   7.4.[Regresion logistica](#Regresion-logistica)
       
8. [Conclusión general](#Conclusión-general)

## Inicializacion
Abre el archivo de datos y estudia la información general


In [1]:
# Cargar todas las librerías
import pandas as pd 
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


In [2]:
df_churn = pd.read_csv('/datasets/Churn.csv',sep=',', header=0)

## Informacion general
sobre el DataFrame users_behavior

In [3]:
def first_lookup(datos):
    print('Primera filas:')
    print(datos.head())
    print()
    print('Informacion:')
    print(datos.info())
    print()
    print('El total de valores ausentes es:')
    print(datos.isna().sum())
    print()
    print('El total de valores duplicados es:')
    print(datos.duplicated().sum())
print(first_lookup(df_churn))

Primera filas:
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0 

In [4]:
# Calcular el porcentaje de valores ausentes en la columna 'Tenure'
missing_percentage = (df_churn['Tenure'].isna().sum() / len(df_churn)) * 100
# Imprimir el resultado
print(f"Porcentaje de valores ausentes en 'Tenure': {missing_percentage:.2f}%")

Porcentaje de valores ausentes en 'Tenure': 9.09%


In [5]:
# Imputar los valores ausentes en la columna 'Tenure' con la media
df_churn['Tenure'].fillna(df_churn['Tenure'].mean(), inplace=True)
#Verificar que ya no haya valores ausentes
tenure_after_imputation = (df_churn['Tenure'].isna().sum() / len(df_churn)) * 100
print(f"Porcentaje de valores ausentes en 'Tenure' después de la imputación: {tenure_after_imputation:.2f}%")

Porcentaje de valores ausentes en 'Tenure' después de la imputación: 0.00%


In [6]:
churn_clean= df_churn.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
print(churn_clean.head())

   CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          619    France  Female   42     2.0       0.00              1   
1          608     Spain  Female   41     1.0   83807.86              1   
2          502    France  Female   42     8.0  159660.80              3   
3          699    France  Female   39     1.0       0.00              2   
4          850     Spain  Female   43     2.0  125510.82              1   

   HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0          1               1        101348.88       1  
1          0               1        112542.58       0  
2          1               0        113931.57       1  
3          0               0         93826.63       0  
4          1               1         79084.10       0  


In [7]:
churn_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


RowNumber: índice de cadena de datos,CustomerId: identificador de cliente único ,Surname: apellido estan columnas son eliminadas porque no me ayudar en mi moemnto de predecir cuantos cliente se van a ir o se van quedar, apellido no me sirve ya que cuento con el genero, al igual que identificador del cliente, ni tampoco el indice de cadena de datos, las demas columnas si son relevantes para poder hacer mi analisis escoger el modelo correcto de prediccion

In [8]:
churn_clean = pd.get_dummies(churn_clean, drop_first=True, columns=['Geography', 'Gender'])
churn_clean.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


In [9]:
#Preparar las caracteristicas y el objetivo
features = churn_clean.drop(['Exited'], axis=1)
target = churn_clean['Exited']
print(type(target))

<class 'pandas.core.series.Series'>


##  Segmenta los datos en entrenamiento, validacion y prueba.

In [10]:
# Dividir los datos en conjuntos de entrenamiento (60%), validación (20%) y prueba (20%)
features_train, features_temp, target_train, target_temp =train_test_split(features, target, test_size= 0.4, random_state=12345) # 40% para validación y prueba)
features_valid, features_test, target_valid, target_test= train_test_split(features_temp, target_temp, test_size=0.5, random_state=12345 )# Divide el 40% en validación y prueba


In [11]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(6000, 11)
(6000,)
(2000, 11)
(2000,)
(2000, 11)
(2000,)


In [12]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

In [13]:
# Guardar los nombres de las columnas antes de la estandarización
column_names = numeric

In [14]:
## Estandarizar las caracteristicas numericas
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train[numeric]) 
features_valid = scaler.fit_transform(features_valid[numeric])
features_test = scaler.fit_transform(features_test[numeric])
print(features_train)
print(features_valid)
print(features_test)


[[-0.88675067 -0.37319167  1.08235388 ...  0.64246552 -1.05518729
  -0.18770538]
 [ 0.60866264 -0.18338494  1.08235388 ... -1.55650375 -1.05518729
  -0.33394547]
 [ 2.05215189  0.48093862 -0.73761762 ... -1.55650375  0.94769906
   1.50309521]
 ...
 [ 0.02711302  0.57584199 -0.01046984 ... -1.55650375  0.94769906
  -0.90315791]
 [ 0.1517308  -1.41712869 -0.37362332 ... -1.55650375  0.94769906
  -1.12853893]
 [ 0.40096635 -0.08848157 -1.46560622 ... -1.55650375 -1.05518729
  -0.9498415 ]]
[[-0.64373967 -0.36630504 -1.02426456 ...  0.64764683 -0.99501244
  -0.0432994 ]
 [-0.22910255  0.56790827 -0.6641527  ...  0.64764683  1.00501256
   1.43924509]
 [ 0.20626642 -0.64656903 -1.74448828 ...  0.64764683 -0.99501244
  -1.39127552]
 ...
 [-1.30715905  0.00738029  0.41618287 ... -1.54405141  1.00501256
  -1.27360081]
 [ 1.12883399  2.62317754 -0.30404085 ...  0.64764683  1.00501256
  -0.13895192]
 [-0.38459147  0.94159359  0.41618287 ...  0.64764683 -0.99501244
   1.6350627 ]]
[[-2.15234333 -0

In [15]:
# Verificar el equilibrio de clases
class_distribution = target.value_counts(normalize=True)
print('Class distribution')
print(class_distribution)

Class distribution
0    0.7963
1    0.2037
Name: Exited, dtype: float64


## Entrena el modelo sin tener en cuenta el desequilibrio. Describe brevemente tus hallazgos
a) Arbol de decision
b) Bosque aleatorio
c) Regresion logistica

### Modelo Arbol de decision

In [16]:
## Crear y entrenar el modelo de Arbol de decision
dt_model = DecisionTreeClassifier(max_depth=7, min_samples_split=0.05, random_state=12345, class_weight='balanced')
dt_model.fit(features_train, target_train)
## Hacer predicciones en el conjunto de entraniento y de  validacion
predictions_valid_tree = dt_model.predict(features_valid)
predictions_train_tree = dt_model.predict(features_train)
#Calcular metricas de F1 y AUC-ROC
f1_valid_tree = f1_score(target_valid, predictions_valid_tree)
f1_train_tree = f1_score(target_train, predictions_train_tree)
auc_roc_valid_tree = roc_auc_score(target_valid, predictions_valid_tree)
auc_roc_train_tree = roc_auc_score(target_train, predictions_train_tree)
print('Arbol de decision-entranamiento-F1:', f1_train_tree)
print('Arbol de decision-validacion-F1:', f1_valid_tree)
print('Arbol de decision-entrenamiento-AUC-ROC:', auc_roc_train_tree)
print('Arbol de decision-validacion-AUC-ROC:', auc_roc_valid_tree)

Arbol de decision-entranamiento-F1: 0.5612767238783499
Arbol de decision-validacion-F1: 0.5637823371989295
Arbol de decision-entrenamiento-AUC-ROC: 0.7654647464905222
Arbol de decision-validacion-AUC-ROC: 0.7556769034412257


a ) El valor de f1 para entrenamiento no llega al objetivo indicado lo que nos dice que el modelo tiene dificultades para identificar correctamente los clientes que abandonanm aunque entre F1 score de entrenamiento y validacion no hay diferencia que nos indique un subajuste ni sobreajuste
b) El valor de AUC-ROC nos indica que el modelo tiene una buena capacidad de discriminar datos no vistos

### Modelo Bosque aletorio

In [17]:
## Crear y entrenar el modelo de Random Forest
rf_model= RandomForestClassifier(n_estimators=100, max_depth=6, random_state=12345, class_weight='balanced')
rf_model.fit(features_train, target_train)
## Hacer predicciones en el conjunto de entrenamiento y  validacion
predictions_train_rf = rf_model.predict(features_train)
predictions_valid_rf = rf_model.predict(features_valid)
#Calcular metricas de F1 y AUC-ROC
f1_train_rf = f1_score(target_train, predictions_train_rf)
f1_valid_rf = f1_score(target_valid, predictions_valid_rf)
auc_roc_train_rf = roc_auc_score(target_train, predictions_train_rf)
auc_roc_valid_rf = roc_auc_score(target_valid, predictions_valid_rf)
print('Bosque aleatorio-train-F1:', f1_train_rf)
print('Bosque aleatorio-valid-F1:', f1_valid_rf)
print('Bosque aleatorio-train-AUC-ROC:', auc_roc_train_rf)
print('Bosque aleatorio-valid-AUC-ROC:', auc_roc_valid_rf)

Bosque aleatorio-train-F1: 0.5987438939288207
Bosque aleatorio-valid-F1: 0.6020408163265307
Bosque aleatorio-train-AUC-ROC: 0.7741827462621729
Bosque aleatorio-valid-AUC-ROC: 0.768483961311162


a) Los valores de F1 score entre entrenamiento y validacion llegaron el objetivo no se nota subajuste ni sobreajuste, el modelo nos indica que tiene buena capacidad para generalizar datos no visto 
b) El valor de AUC_ROC pra entrenamiento y validacion nos indica que el modelo es efectivo para la clasificacion incluso para datos no visto 
c) El desequilibrio de clases nos indica que el modelo esta sesgado hacia la clase mayoritaria esto nos lleva a un rendimiento no optimo para la identificacion de la clase minoritaria

### Modelo Regresion logisitca

In [18]:
## Crear y entrenar el modelo de Regresion Logistica
logistic_model = LogisticRegression(solver='liblinear', random_state=12345, class_weight='balanced')
#Entrenar el modelo
logistic_model.fit(features_train, target_train)
## Hacer predicciones en el conjunto de entrenamiento y  validacion
predictions_train_lm = logistic_model.predict(features_train)
predictions_valid_lm = logistic_model.predict(features_valid)
#Calcular metricas de F1 y AUC-ROC
f1_train_lm = f1_score(target_train, predictions_train_lm)
f1_valid_lm = f1_score(target_valid, predictions_valid_lm)
auc_roc_train_lm = roc_auc_score(target_train, predictions_train_lm)
auc_roc_valid_lm = roc_auc_score(target_valid, predictions_valid_lm)
print('Regresion logistica-train-F1:', f1_train_lm)
print('Regresion logistica-valid-F1:', f1_valid_lm)
print('Regresion logistica-train-AUC-ROC:', auc_roc_train_lm)
print('Regresion logistica-valid-AUC-ROC:', auc_roc_valid_lm)


Regresion logistica-train-F1: 0.47624499141385235
Regresion logistica-valid-F1: 0.4793103448275862
Regresion logistica-train-AUC-ROC: 0.6952449046084784
Regresion logistica-valid-AUC-ROC: 0.6858860748008396


a) El valor de F1 tanto para entrenamiento y validacion son demasiado bajos y no llegan al objetivo deseado  el modelo tiene difilcutades para generalizar datos no vistos  y no es efectivo  para la identificacion de clientes en riesgo de abandono
b) Los valores de AUC-ROC para el conjunto de entrenamiento y validacion nos indican que el modelo tiene una capacidad limitada para discriminar correctamente entre las clases cuando se enfrenta a datos no vistos.

## Mejora la calidad del modelo. Asegúrate de utilizar al menos dos enfoques para corregir el desequilibrio de clases. Utiliza conjuntos de entrenamiento y validación para encontrar el mejor modelo y el mejor conjunto de parámetros. Entrena diferentes modelos en los conjuntos de entrenamiento y validación. Encuentra el mejor. Describe brevemente tus hallazgos.

### Aplicacion del sobremuestreo para los modelos de Regresion logistica, Bosque aleatorio y Arbol de decision

In [19]:
# Verificar tipos antes del sobremuestreo
print("Tipo de features_train:", type(features_train))  # Debe ser un DataFrame
print("Tipo de target_train:", type(target_train))      # Debe ser una Series

Tipo de features_train: <class 'numpy.ndarray'>
Tipo de target_train: <class 'pandas.core.series.Series'>


In [20]:
# Convertir features_train a DataFrame y mantener los nombres de las columnas
features_train = pd.DataFrame(features_train, columns=column_names)
features_valid = pd.DataFrame(features_valid, columns=column_names)
# Verificar tipos despues de la conversion
print("Tipo de features_train despues de la conversion:", type(features_train))  # Ahora debe ser un DataFrame
# Verificar tipos despues de la conversion
print("Tipo de features_valid después de la conversion:", type(features_valid))  # Ahora debe ser un DataFrame

Tipo de features_train despues de la conversion: <class 'pandas.core.frame.DataFrame'>
Tipo de features_valid después de la conversion: <class 'pandas.core.frame.DataFrame'>


In [21]:
# Reiniciar índices para asegurar alineación
features_train.reset_index(drop=True, inplace=True)
target_train.reset_index(drop=True, inplace=True)


In [22]:
def upsample(features, target, repeat):
    # Separar las caracteristicas y el objetivo en clases
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target ==1]
    # Crear un nuevo conjunto con mas ejemplos de clase minorittaria
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] *repeat)
    #Barajear los datos para mezclar las clases
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled


In [23]:
# Aplicar sobremuestreo de los datos de entrenamiento
features_upsampled, target_upsampled = upsample(features_train, target_train, repeat=10)

### Arbol de decision

In [24]:
# Inicializar variables para almacenar los mejores resultados
best_f1_score = 0
best_depth = 0 
for depth in range(2, 6):
    dtree_model = DecisionTreeClassifier(random_state=12345, min_samples_split=10, min_samples_leaf=5, max_depth=depth, class_weight='balanced')
    dtree_model.fit(features_upsampled, target_upsampled)
    # Hacer predicciones en el conjunto de entrenamiento y de  validacion
    predictions_train_dtree = dtree_model.predict(features_upsampled)
    predictions_valid_dtree = dtree_model.predict(features_valid)
    # Calcular las metricas de F1 y AUC-ROC
    f1_train_dtree = f1_score(target_upsampled, predictions_train_dtree)
    f1_valid_dtree = f1_score(target_valid, predictions_valid_dtree)
    print(f"Arbol de Decision-train- max_depth: {depth} - F1 Score: {f1_train_dtree:.2f}")
    print(f"Arbol de Decision-valid- max_depth: {depth} - F1 Score: {f1_valid_dtree:.2f}")
    # Actualizar los mejores parametros si se encuentra una mejor puntuacion F1
    if f1_valid_dtree > best_f1_score:
        best_f1_score = f1_valid_dtree
        best_depth = depth
    # Imprimir los mejores parámetros encontrados y la mejor puntuación F1
print('Mejor profundidad encontrada:', best_depth)
print('Mejor F1 Score:', best_f1_score)


Arbol de Decision-train- max_depth: 2 - F1 Score: 0.75
Arbol de Decision-valid- max_depth: 2 - F1 Score: 0.54
Arbol de Decision-train- max_depth: 3 - F1 Score: 0.75
Arbol de Decision-valid- max_depth: 3 - F1 Score: 0.54
Arbol de Decision-train- max_depth: 4 - F1 Score: 0.82
Arbol de Decision-valid- max_depth: 4 - F1 Score: 0.53
Arbol de Decision-train- max_depth: 5 - F1 Score: 0.78
Arbol de Decision-valid- max_depth: 5 - F1 Score: 0.59
Mejor profundidad encontrada: 5
Mejor F1 Score: 0.591880341880342


In [25]:
# Evaluar AUC-ROC para el Árbol de Decisión usando probabilidades
probabilities_train = dtree_model.predict_proba(features_upsampled)[:, 1]
probabilities_valid = dtree_model.predict_proba(features_valid)[:, 1]

auc_roc_train_dtree = roc_auc_score(target_upsampled, probabilities_train)
auc_roc_valid_dtree = roc_auc_score(target_valid, probabilities_valid)

print('Arbol de Decisión-train- AUC-ROC:', auc_roc_train_dtree)
print('Arbol de Decisión-valid- AUC-ROC:', auc_roc_valid_dtree)

# Ajuste del umbral 
precision, recall, thresholds = precision_recall_curve(target_valid, probabilities_valid)

# Calcular F1 Score para cada umbral
f1_scores = 2 * (precision * recall) / (precision + recall)

# Encontrar el umbral que da el mejor F1 Score
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
print("Mejor umbral:", best_threshold)

# Hacer predicciones con el nuevo umbral ajustado
predictions_adjusted_train = (probabilities_train >= best_threshold).astype(int)
predictions_adjusted_valid = (probabilities_valid >= best_threshold).astype(int)

# Calcular el nuevo F1 Score ajustado para entrenamiento y validación
f1_adjusted_train = f1_score(target_upsampled, predictions_adjusted_train)
f1_adjusted_valid = f1_score(target_valid, predictions_adjusted_valid)

print("F1 Score ajustado (entrenamiento):", f1_adjusted_train)
print("F1 Score ajustado (validación):", f1_adjusted_valid)

# Evaluar AUC-ROC con el nuevo umbral ajustado para validación y entrenamiento usando probabilidades
auc_roc_adjusted_train = roc_auc_score(target_upsampled, predictions_adjusted_train)
auc_roc_adjusted_valid = roc_auc_score(target_valid, predictions_adjusted_valid)

print('AUC-ROC con umbral ajustado (entrenamiento):', auc_roc_adjusted_train)
print('AUC-ROC con umbral ajustado (validación):', auc_roc_adjusted_valid)

Arbol de Decisión-train- AUC-ROC: 0.849037887184314
Arbol de Decisión-valid- AUC-ROC: 0.8241551182864644
Mejor umbral: 0.6647601476014764
F1 Score ajustado (entrenamiento): 0.7637629295553704
F1 Score ajustado (validación): 0.6034285714285715
AUC-ROC con umbral ajustado (entrenamiento): 0.7581210195517114
AUC-ROC con umbral ajustado (validación): 0.7547907379067137


 a) El valor de F1 de 0.4911 un valor alejado al objetivo del proyecto de llegar a un F1 de 0.59 nos indican que el modelo no esta capturando bien la relaciones de los datos puede indicar sobreajuste o subajuste y en comparacion con el valor de f1 de arbol de decision con desequilibrios de clases
b) El valor de AUC-ROC : 0.6760 con un valor menor en comparacion el arbol de decision con desequilibrio de clase, esta lejos de ser optimo

### Bosque Aleatorio

In [26]:
# Inicializar variables para almacenar los mejores parametros y la mejor puntuacion F1
best_f1_score = 0
best_params = {}

# Probar diferentes combinaciones de hiperparametros
for est in range(10, 61, 10):  
    for depth in range(3, 5):       
        random_forest = RandomForestClassifier(n_estimators=est, min_samples_split=10, min_samples_leaf=5, max_depth=depth, random_state=12345, class_weight='balanced')
        
        # Entrenar el modelo con los datos submuestreados
        random_forest.fit(features_upsampled, target_upsampled)
        
        # Realizar predicciones con el conjunto de entrenamiento y validacion
        predictions_train_randomforest = random_forest.predict(features_upsampled)
        predictions_valid_randomforest = random_forest.predict(features_valid)
        
        # Calcular el valor F1 del modelo
        f1_train_random_forest = f1_score(target_upsampled, predictions_train_randomforest)
        f1_valid_random_forest = f1_score(target_valid, predictions_valid_randomforest)
        
        print(f"Random Forest-train- n_estimators: {est}, max_depth: {depth} - F1 Score: {f1_train_random_forest:.2f}")
        print(f"Random Forest-valid- n_estimators: {est}, max_depth: {depth} - F1 Score: {f1_valid_random_forest:.2f}")
        # Actualizar los mejores parametros si se encuentra una mejor puntuacion F1
        if f1_valid_random_forest > best_f1_score:
            best_f1_score = f1_valid_random_forest
            best_params = {'n_estimators': est, 'max_depth': depth}

# Imprimir los mejores parametros encontrados y la mejor puntuacion F1
print('Mejores parametros encontrados:')
print(best_params)
print('Mejor F1 Score:', best_f1_score)


Random Forest-train- n_estimators: 10, max_depth: 3 - F1 Score: 0.75
Random Forest-valid- n_estimators: 10, max_depth: 3 - F1 Score: 0.56
Random Forest-train- n_estimators: 10, max_depth: 4 - F1 Score: 0.76
Random Forest-valid- n_estimators: 10, max_depth: 4 - F1 Score: 0.58
Random Forest-train- n_estimators: 20, max_depth: 3 - F1 Score: 0.76
Random Forest-valid- n_estimators: 20, max_depth: 3 - F1 Score: 0.57
Random Forest-train- n_estimators: 20, max_depth: 4 - F1 Score: 0.78
Random Forest-valid- n_estimators: 20, max_depth: 4 - F1 Score: 0.59
Random Forest-train- n_estimators: 30, max_depth: 3 - F1 Score: 0.75
Random Forest-valid- n_estimators: 30, max_depth: 3 - F1 Score: 0.58
Random Forest-train- n_estimators: 30, max_depth: 4 - F1 Score: 0.78
Random Forest-valid- n_estimators: 30, max_depth: 4 - F1 Score: 0.59
Random Forest-train- n_estimators: 40, max_depth: 3 - F1 Score: 0.76
Random Forest-valid- n_estimators: 40, max_depth: 3 - F1 Score: 0.58
Random Forest-train- n_estimators:

In [27]:
probabilities_train = random_forest.predict_proba(features_upsampled)[:, 1]
probabilities_valid = random_forest.predict_proba(features_valid)[:, 1]

auc_roc_train_randomforest = roc_auc_score(target_upsampled, probabilities_train)
auc_roc_valid_randomforest = roc_auc_score(target_valid, probabilities_valid)

print('Bosque aleatorio - AUC-ROC (entrenamiento):', auc_roc_train_randomforest)
print('Bosque aleatorio - AUC-ROC (validación):', auc_roc_valid_randomforest)

# Ajuste del umbral 
precision, recall, thresholds = precision_recall_curve(target_valid, probabilities_valid)

# Calcular F1 Score para cada umbral
f1_scores = 2 * (precision * recall) / (precision + recall)

# Encontrar el umbral que da el mejor F1 Score
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
print("Mejor umbral:", best_threshold)

# Hacer predicciones con el nuevo umbral ajustado
predictions_adjusted_train = (probabilities_train >= best_threshold).astype(int)
predictions_adjusted_valid = (probabilities_valid >= best_threshold).astype(int)

# Calcular el nuevo F1 Score ajustado para entrenamiento y validación
f1_adjusted_train = f1_score(target_upsampled, predictions_adjusted_train)
f1_adjusted_valid = f1_score(target_valid, predictions_adjusted_valid)

print("F1 Score ajustado (entrenamiento):", f1_adjusted_train)
print("F1 Score ajustado (validación):", f1_adjusted_valid)

# Evaluar AUC-ROC con el nuevo umbral ajustado para validación y entrenamiento usando probabilidades
auc_roc_adjusted_train = roc_auc_score(target_upsampled, predictions_adjusted_train)
auc_roc_adjusted_valid = roc_auc_score(target_valid, predictions_adjusted_valid)

print('AUC-ROC con umbral ajustado (entrenamiento):', auc_roc_adjusted_train)
print('AUC-ROC con umbral ajustado (validación):', auc_roc_adjusted_valid)

Bosque aleatorio - AUC-ROC (entrenamiento): 0.8475879040320358
Bosque aleatorio - AUC-ROC (validación): 0.8343324118824818
Mejor umbral: 0.5805869975613694
F1 Score ajustado (entrenamiento): 0.7169752927219271
F1 Score ajustado (validación): 0.617283950617284
AUC-ROC con umbral ajustado (entrenamiento): 0.740905711238405
AUC-ROC con umbral ajustado (validación): 0.7541631633387573


### Regresion logistica

In [28]:
# Inicializar el modelo de Regresion logistica
log_model = LogisticRegression(solver='liblinear', random_state=54321, class_weight= 'balanced')
#Entrenar el modelo
log_model.fit(features_upsampled, target_upsampled)
#Realizar predicciones del conjunto de enrenamiento y  validacion
predictions_log_train = log_model.predict(features_upsampled)
predictions_log_valid = log_model.predict(features_valid)
#Calcular metricas de F1 y AUC-ROC
f1_log_train = f1_score(target_upsampled, predictions_log_train)
f1_log_valid = f1_score(target_valid, predictions_log_valid)
auc_roc_log_train = roc_auc_score(target_upsampled, predictions_log_train)
auc_roc_log_valid = roc_auc_score(target_valid, predictions_log_valid)
print('Regresion logistica-train-F1:', f1_log_train)
print('Regresion logistica-valid-F1:', f1_log_valid)
print('Regresion logistica-train-AUC-ROC:', auc_roc_log_train)
print('Regresion logistica-valid-AUC-ROC:', auc_roc_log_valid)

Regresion logistica-train-F1: 0.7652333869855139
Regresion logistica-valid-F1: 0.4793103448275862
Regresion logistica-train-AUC-ROC: 0.6953489845418673
Regresion logistica-valid-AUC-ROC: 0.6858860748008396


## Aplicacion del submuestro para los modelos de Regresion logistica, Bosque aleatorio y Arbol de decision

In [29]:
def downsample(features, target, fraction):
    #Separar las caracteristicas y el objetivo en clases
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    #Crear un nuevo conjunto  con mas ejemplos de la clase mayoritaria
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    # Barajar los datos para mezclar las clases
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

In [30]:
# Aplicar submuestreo a los datos de entrenamiento
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

### Arbol de decision

In [31]:
# Inicializar variables para almacenar los mejores resultados
best_f1_score = 0
best_depth = 0 
# Probar diferentes profundidades para el arbol de decision
for depth in range(2, 5):
    decision_tree_model = DecisionTreeClassifier(random_state=12345, min_samples_split=10, min_samples_leaf=5, max_depth=depth, class_weight='balanced')
    decision_tree_model.fit(features_downsampled, target_downsampled)
    # Hacer predicciones en el conjunto de entrenamiento y  validacion
    predictions_decision_tree_train = decision_tree_model.predict(features_downsampled)
    predictions_decision_tree_valid = decision_tree_model.predict(features_valid)
    # Calcular las metricas de F1 y AUC-ROC
    f1_decision_tree_train = f1_score(target_downsampled, predictions_decision_tree_train)
    f1_decision_tree_valid = f1_score(target_valid, predictions_decision_tree_valid)
    print(f"Arbol de Decision-train - max_depth: {depth} - F1 Score: {f1_decision_tree_train:.2f}")
    print(f"Arbol de Decision-valid - max_depth: {depth} - F1 Score: {f1_decision_tree_valid:.2f}")
    # Actualizar los mejores parametros si se encuentra una mejor puntuacion F1
    if f1_decision_tree_valid > best_f1_score:
        best_f1_score = f1_decision_tree_valid
        best_depth = depth
    # Imprimir los mejores parametros encontrados y la mejor puntuacion F1
print('Mejor profundidad encontrada:', best_depth)
print('Mejor F1 Score:', best_f1_score)


Arbol de Decision-train - max_depth: 2 - F1 Score: 0.81
Arbol de Decision-valid - max_depth: 2 - F1 Score: 0.49
Arbol de Decision-train - max_depth: 3 - F1 Score: 0.79
Arbol de Decision-valid - max_depth: 3 - F1 Score: 0.52
Arbol de Decision-train - max_depth: 4 - F1 Score: 0.80
Arbol de Decision-valid - max_depth: 4 - F1 Score: 0.54
Mejor profundidad encontrada: 4
Mejor F1 Score: 0.5402504472271915


In [32]:
# Evaluar AUC-ROC para el Árbol de Decisión usando probabilidades
probabilities_train = decision_tree_model.predict_proba(features_downsampled)[:, 1]
probabilities_valid = decision_tree_model.predict_proba(features_valid)[:, 1]

auc_roc_decision_tree_train = roc_auc_score(target_downsampled, probabilities_train)
auc_roc_decision_tree_valid = roc_auc_score(target_valid, probabilities_valid)

print('Arbol de Decisión-train - AUC-ROC:', auc_roc_decision_tree_train)
print('Arbol de Decisión-valid - AUC-ROC:', auc_roc_decision_tree_valid)

# Ajuste del umbral 
precision, recall, thresholds = precision_recall_curve(target_valid, probabilities_valid)

# Calcular F1 Score para cada umbral
f1_scores = 2 * (precision * recall) / (precision + recall)

# Encontrar el umbral que da el mejor F1 Score
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
print("Mejor umbral:", best_threshold)

# Hacer predicciones con el nuevo umbral ajustado
predictions_adjusted_train = (probabilities_train >= best_threshold).astype(int)
predictions_adjusted_valid = (probabilities_valid >= best_threshold).astype(int)

# Calcular el nuevo F1 Score ajustado para entrenamiento y validación
f1_adjusted_train = f1_score(target_downsampled, predictions_adjusted_train)
f1_adjusted_valid = f1_score(target_valid, predictions_adjusted_valid)

print("F1 Score ajustado (entrenamiento):", f1_adjusted_train)
print("F1 Score ajustado (validación):", f1_adjusted_valid)

# Evaluar AUC-ROC con el nuevo umbral ajustado para validación y entrenamiento usando probabilidades
auc_roc_adjusted_train = roc_auc_score(target_downsampled, predictions_adjusted_train)
auc_roc_adjusted_valid = roc_auc_score(target_valid, predictions_adjusted_valid)
print("AUC-ROC ajustado (entrenamiento):", auc_roc_adjusted_train)
print("AUC-ROC ajustado (validación):", auc_roc_adjusted_valid)


Arbol de Decisión-train - AUC-ROC: 0.8223766722408026
Arbol de Decisión-valid - AUC-ROC: 0.7944867196148054
Mejor umbral: 0.6801265563471662
F1 Score ajustado (entrenamiento): 0.782319391634981
F1 Score ajustado (validación): 0.5444340505144996
AUC-ROC ajustado (entrenamiento): 0.7555218784838349
AUC-ROC ajustado (validación): 0.7343060991174638


En el caso de submuestreo se ajusto el F1 score tanto para valores de entrenamiento y validacion  hay una marcada difrencia de valores y un marcado sobreajuste entre ellos, ya que el conjunto de validacion no llega al objetivo de 0.59 no es optimo para poderle realizar la prueba final
b) La diferencia entre AUC-ROC entre entrenamiento y validacion nos dicedisminución puede ser un indicativo de que el modelo esta comenzando a perder algo de efectividad al generalizar a datos no vistos.

### Bosque aleatorio

In [33]:
# Inicializar variables para almacenar los mejores parametros y la mejor puntuacion F1
best_f1_score = 0
best_params = {}

# Probar diferentes combinaciones de hiperparametros
for est in range(10, 101, 10):  
    for depth in range(1, 11):       
        randomforest_model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345, class_weight='balanced')
        
        # Entrenar el modelo con los datos submuestreados
        randomforest_model.fit(features_downsampled, target_downsampled)
        
        # Realizar predicciones con el conjunto de entrenamiento y validacion
        predictions_randomforest_train = randomforest_model.predict(features_downsampled)
        predictions_randomforest_valid = randomforest_model.predict(features_valid)
        
        # Calcular el valor F1 del modelo
        f1_randomforest_train = f1_score(target_downsampled, predictions_randomforest_train)
        f1_randomforest_valid = f1_score(target_valid, predictions_randomforest_valid)
        
        print(f"Random Forest-train- n_estimators: {est}, max_depth: {depth} - F1 Score: {f1_randomforest_train:.2f}")
        print(f"Random Forest-valid- n_estimators: {est}, max_depth: {depth} - F1 Score: {f1_randomforest_valid:.2f}")
        
        # Actualizar los mejores parametros si se encuentra una mejor puntuacion F1
        if f1_randomforest_valid > best_f1_score:
            best_f1_score = f1_randomforest_valid
            best_params = {'n_estimators': est, 'max_depth': depth}

# Imprimir los mejores parametros encontrados y la mejor puntuacion F1
print('Mejores parámetros encontrados:')
print(best_params)
print('Mejor F1 Score:', best_f1_score)


Random Forest-train- n_estimators: 10, max_depth: 1 - F1 Score: 0.77
Random Forest-valid- n_estimators: 10, max_depth: 1 - F1 Score: 0.54
Random Forest-train- n_estimators: 10, max_depth: 2 - F1 Score: 0.77
Random Forest-valid- n_estimators: 10, max_depth: 2 - F1 Score: 0.54
Random Forest-train- n_estimators: 10, max_depth: 3 - F1 Score: 0.78
Random Forest-valid- n_estimators: 10, max_depth: 3 - F1 Score: 0.54
Random Forest-train- n_estimators: 10, max_depth: 4 - F1 Score: 0.80
Random Forest-valid- n_estimators: 10, max_depth: 4 - F1 Score: 0.55
Random Forest-train- n_estimators: 10, max_depth: 5 - F1 Score: 0.81
Random Forest-valid- n_estimators: 10, max_depth: 5 - F1 Score: 0.53
Random Forest-train- n_estimators: 10, max_depth: 6 - F1 Score: 0.83
Random Forest-valid- n_estimators: 10, max_depth: 6 - F1 Score: 0.54
Random Forest-train- n_estimators: 10, max_depth: 7 - F1 Score: 0.85
Random Forest-valid- n_estimators: 10, max_depth: 7 - F1 Score: 0.55
Random Forest-train- n_estimators:

Random Forest-train- n_estimators: 70, max_depth: 2 - F1 Score: 0.78
Random Forest-valid- n_estimators: 70, max_depth: 2 - F1 Score: 0.54
Random Forest-train- n_estimators: 70, max_depth: 3 - F1 Score: 0.80
Random Forest-valid- n_estimators: 70, max_depth: 3 - F1 Score: 0.55
Random Forest-train- n_estimators: 70, max_depth: 4 - F1 Score: 0.81
Random Forest-valid- n_estimators: 70, max_depth: 4 - F1 Score: 0.55
Random Forest-train- n_estimators: 70, max_depth: 5 - F1 Score: 0.82
Random Forest-valid- n_estimators: 70, max_depth: 5 - F1 Score: 0.56
Random Forest-train- n_estimators: 70, max_depth: 6 - F1 Score: 0.84
Random Forest-valid- n_estimators: 70, max_depth: 6 - F1 Score: 0.55
Random Forest-train- n_estimators: 70, max_depth: 7 - F1 Score: 0.87
Random Forest-valid- n_estimators: 70, max_depth: 7 - F1 Score: 0.54
Random Forest-train- n_estimators: 70, max_depth: 8 - F1 Score: 0.90
Random Forest-valid- n_estimators: 70, max_depth: 8 - F1 Score: 0.54
Random Forest-train- n_estimators:

In [34]:
# Evaluar AUC-ROC para el Bosque Aleatorio usando probabilidades
probabilities_train = randomforest_model.predict_proba(features_downsampled)[:, 1]
probabilities_valid = randomforest_model.predict_proba(features_valid)[:, 1]

auc_roc_randomforest_model_train = roc_auc_score(target_downsampled, probabilities_train)
auc_roc_randomforest_model_valid = roc_auc_score(target_valid, probabilities_valid)

print('Bosque aleatorio - AUC-ROC (entrenamiento):', auc_roc_randomforest_model_train)
print('Bosque aleatorio - AUC-ROC (validación):', auc_roc_randomforest_model_valid)

# Ajuste del umbral 
precision, recall, thresholds = precision_recall_curve(target_valid, probabilities_valid)

# Calcular F1 Score para cada umbral
f1_scores = 2 * (precision * recall) / (precision + recall)

# Encontrar el umbral que da el mejor F1 Score
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
print("Mejor umbral:", best_threshold)

# Hacer predicciones con el nuevo umbral ajustado
predictions_adjusted_train = (probabilities_train >= best_threshold).astype(int)
predictions_adjusted_valid = (probabilities_valid >= best_threshold).astype(int)

# Calcular el nuevo F1 Score ajustado para entrenamiento y validacion
f1_adjusted_train = f1_score(target_downsampled, predictions_adjusted_train)
f1_adjusted_valid = f1_score(target_valid, predictions_adjusted_valid)

print("F1 Score ajustado (entrenamiento):", f1_adjusted_train)
print("F1 Score ajustado (validación):", f1_adjusted_valid)

# Evaluar AUC-ROC con el nuevo umbral ajustado para validacion y entrenamiento usando probabilidades
auc_roc_adjusted_train = roc_auc_score(target_downsampled, predictions_adjusted_train)
auc_roc_adjusted_valid = roc_auc_score(target_valid, predictions_adjusted_valid)

print('AUC-ROC con umbral ajustado (entrenamiento):', auc_roc_adjusted_train)
print('AUC-ROC con umbral ajustado (validación):', auc_roc_adjusted_valid)

Bosque aleatorio - AUC-ROC (entrenamiento): 0.988980629877369
Bosque aleatorio - AUC-ROC (validación): 0.8220606826801518
Mejor umbral: 0.6929203496213145
F1 Score ajustado (entrenamiento): 0.8722300801508722
F1 Score ajustado (validación): 0.5883597883597883
AUC-ROC con umbral ajustado (entrenamiento): 0.8867056856187291
AUC-ROC con umbral ajustado (validación): 0.7538380343457195


Para el valor de F1 score ajustado entre entrenamiento y validacion nos indica un muy notorio sobreajuste el modelo tiene dificultades para identificar correctamente la clase positiva en datos no vistos
b) Para la comparacion de AUC-ROC ajustado  para el conjunto de entrenamiento y validacion indica una buena capacidad discriminativa  pero refleja una disminucion en comparacion con el entrenamiento.

### Regresion logistica


In [35]:
# Inicializar el modelo de Regresion logistica
logistic_reg = LogisticRegression(solver='liblinear', random_state=54321, class_weight= 'balanced')
#Entrenar el modelo
logistic_reg.fit(features_downsampled, target_downsampled)
#Realizar predicciones del conjunto de entramiento y de validacion
logistic_reg_predictions_train = logistic_reg.predict(features_downsampled)
logistic_reg_predictions_valid = logistic_reg.predict(features_valid)
#Calcular metricas de F1 y AUC-ROC
f1_logistic_reg_train = f1_score(target_downsampled, logistic_reg_predictions_train)
f1_logistic_reg_valid = f1_score(target_valid, logistic_reg_predictions_valid)
auc_roc_logistic_reg_train = roc_auc_score(target_downsampled, logistic_reg_predictions_train)
auc_roc_logistic_reg_valid = roc_auc_score(target_valid, logistic_reg_predictions_valid)
print('Regresion logistica-train-F1:', f1_logistic_reg_train)
print('Regresion logistica-valid-F1:', f1_logistic_reg_valid)
print('Regresion logistica-train-AUC-ROC:', auc_roc_logistic_reg_train)
print('Regresion logistica-valid-AUC-ROC:', auc_roc_logistic_reg_valid)

Regresion logistica-train-F1: 0.7782388197325957
Regresion logistica-valid-F1: 0.4802744425385935
Regresion logistica-train-AUC-ROC: 0.7184678093645485
Regresion logistica-valid-AUC-ROC: 0.6870141967952866


a) Para la compracion entre el F1 acore nos indica un sobreajuste notorio este descenso indica que el modelo tiene dificultades para generalizar a datos no vistos y no logra identificar adecuadamente a los clientes que abandonan
b) Para la compracion de AUC-ROC entre el conjunto de entramiento y validacion refleja una capacidad limitada para discriminar correctamente entre las clases en datos no vistos, este  valor es inferior al AUC del conjunto de entrenamiento, lo que nos muestra la idea de que el modelo no se está generalizando correctamente

## Realiza la prueba final.

### Se realizara la prueba final con el sobremuestreo con los modelos de Random Forest y Decision Tree que fueron los modelos con un F1 score con conjuntos de entrenamiento y validacion  mayores o iguales a 0.59 que es el objetivo a llegar en el proyecto

### Decision Tree

In [36]:
# Hacer predicciones en el conjunto de prueba
probabilities_test = dtree_model.predict_proba(features_test)[:, 1]

# Aplicar el umbral ajustado
predictions_adjusted_test = (probabilities_test >= best_threshold).astype(int)

# Calcular F1 Score para el conjunto de prueba
f1_test = f1_score(target_test, predictions_adjusted_test)

# Calcular AUC-ROC para el conjunto de prueba
auc_roc_test = roc_auc_score(target_test, probabilities_test)

print("F1 Score ajustado (prueba):", f1_test)
print("AUC-ROC con umbral ajustado (prueba):", auc_roc_test)

F1 Score ajustado (prueba): 0.514792899408284
AUC-ROC con umbral ajustado (prueba): 0.8244122439740299


In [37]:
# Definir los datos
data = {
    'Métrica': ['F1 Score Ajustado', 'AUC-ROC Ajustado'],
    'Entrenamiento': [0.7638, 0.7581],
    'Validacion': [0.6034, 0.7548],
    'Prueba': [0.514792899408284, 0.8244122439740299]
}

# Crear el DataFrame
metrics_df = pd.DataFrame(data)

# Mostrar el DataFrame
print(metrics_df)

             Métrica  Entrenamiento  Validacion    Prueba
0  F1 Score Ajustado         0.7638      0.6034  0.514793
1   AUC-ROC Ajustado         0.7581      0.7548  0.824412


1. Analisis de F1 score
a) Para entrenamiento el valor nos indica que el modelo tiene buen rendimiento en la identificacion de los clientes que abandonan
b) Para validacion el valor disminuye nos indica que el modelo presenta dificultades para generalizar datos no vistos
c) En el conjunto de prueba el valor cae considerablemente por debajo del umbral deseado el modelo no esta identificado adecuadamente a los clientes que abandonan en datos no vistos esto es un claro ejemplo de sobreajuste, pero en compracion con el submuestreo los valores de F1 del conjunto de validacion y entrenamiento ni siquiera llegaban al valor de 0.59, con sobremuestreo y ajustando el umbral mejorares los valores para F1.
2. AUC-ROC
a) Para entrenamiento el valor nos indica que el modelo tiene capacidad de discriminar entre las personas que abandonan y las que no.
b) Para validacion presenta un valor adecuado el cual nos sugiere que el modelo es efectivo en la clasificacion
c) Para prueba aumenta en valor en compracion nos indica que el modelo tiene la capacidad para clasificar correctamente las clases
Compracion entre F1 score y AUC-ROC
aunque el modelo puede discriminar bien entre las clases mencionadas no presenta una armonia o equilibrio adecuado entre la precision y el recall en la identificacion de los clientes que abandonan 


### Random Forest

In [38]:
# Hacer predicciones en el conjunto de prueba
probabilities_test = random_forest.predict_proba(features_test)[:, 1]

# Aplicar el umbral ajustado
predictions_adjusted_test = (probabilities_test >= best_threshold).astype(int)

# Calcular F1 Score para el conjunto de prueba
f1_test = f1_score(target_test, predictions_adjusted_test)

# Calcular AUC-ROC para el conjunto de prueba
auc_roc_test = roc_auc_score(target_test, probabilities_test)

# Imprimir los resultados
print("F1 Score ajustado (prueba):", f1_test)
print("AUC-ROC con umbral ajustado (prueba):", auc_roc_test)

F1 Score ajustado (prueba): 0.5059171597633136
AUC-ROC con umbral ajustado (prueba): 0.8073106161113286


In [39]:
# Definir los datos
data = {
    'Metrica': ['F1 Score Ajustado', 'AUC-ROC Ajustado'],
    'Entrenamiento': [0.7169, 0.7409],
    'Validacion': [0.6173, 0.7542],
    'Prueba': [0.5059, 0.8073]
}

# Crear el DataFrame
metrics_df = pd.DataFrame(data)

# Mostrar el DataFrame
print(metrics_df)

             Metrica  Entrenamiento  Validacion  Prueba
0  F1 Score Ajustado         0.7169      0.6173  0.5059
1   AUC-ROC Ajustado         0.7409      0.7542  0.8073


1. Analisis de F1 score
a) Para entrenamiento el valor nos indica que el modelo tiende buen rendimiento con los clientes que abandonan
b) Para validacion el resultado nos sugiere que el modelo comienza a presentar dificultades  para generalizar datos no vistos 
c) En el conjunto de prueba el valor nos indica que esta bajo valor deseado de F1 ya que el modelo no esta identificando correctamente los clientes que abandonan en datos no vistos
2. AUC-ROC
a) Para entrenamiento el valor nos indica que el modelo tiene capacidad de discriminar entre las personas que abandonan y las que no.
b) Para validacion en compracion con el entrenamiento aumenta y nos indica la capacidad del modelo para clasificar 
c) Para prueba a pesar de que disminuye el F1 el valor del AUC-ROC nos indica que el modelo sigue teniendo una clasificacion adecuada para la clase que abandona y la que no



### Logistic Regresion

In [40]:
# Hacer predicciones en el conjunto de prueba
predictions_log_test = log_model.predict(features_test)

# Calcular F1 Score para el conjunto de prueba
f1_log_test = f1_score(target_test, predictions_log_test)

# Calcular AUC-ROC para el conjunto de prueba
auc_roc_log_test = roc_auc_score(target_test, predictions_log_test)

# Imprimir los resultados
print("Regresión Logística - F1 Score (Prueba):", f1_log_test)
print("Regresión Logística - AUC-ROC (Prueba):", auc_roc_log_test)

Regresión Logística - F1 Score (Prueba): 0.46351931330472107
Regresión Logística - AUC-ROC (Prueba): 0.6694976996451654


In [41]:
# Definir los datos
data = {
    'Metrica': ['F1 Score', 'AUC-ROC'],
    'Entrenamiento': [0.7652, 0.6953],
    'Validacion': [0.4793, 0.6859],
    'Prueba': [0.4635, 0.6695]
}

# Crear el DataFrame
metrics_df_logistic = pd.DataFrame(data)

# Mostrar el DataFrame
print(metrics_df_logistic)

    Metrica  Entrenamiento  Validacion  Prueba
0  F1 Score         0.7652      0.4793  0.4635
1   AUC-ROC         0.6953      0.6859  0.6695


1. Analisis de F1 score
a) Para entrenamiento el valor indica un rendimiento adecuado para la identificacion de clientes en riesgo de abandonar
b) Para validacion la disminuciion en el valor nos indica que el modelo esta presentando dificultades para generalizar datos no vistos
c) En el conjunto de prueba hay una marcada diferencia en la disminucion del valor lo que nos indica que el modelo sobre la precision en el abandono de clientes
2. AUC-ROC
a) Para entrenamiento el valor nos indicar que el modelo tiene cierta capacidad para diferenciar entre clientes que abandonan y los que no lo hacen
b) Para validacion  el valor disminuye en comparacion, pero a pesar de ello presentando una clafisicando de igual forma efectiva
c) Para prueba el valor nos indica una disminucion en la capacidad del modelo para discriminar correctamente entre las clases en especial con datos no vistos

## Conclusion general

EL objetivo del proyecto desarrollar un modelo predictivo que permita identificar a los clientes en riesgo de abandonar Beta Bank donde modelo se fundamenta en la premisa de que es más costeable retener a los clientes existentes que atraer nuevos, lo que hace esencial la implementación de estrategias efectivas de retencion.
a) En el estudio del modelo donde se presento un desequilibrio de clases notorio, primero se hizo el tratamiento asi para los tres modelo Ramdom Forest, Decision Tree, Regresion Logistic los tres presentaron en F1 score para entrenamiento y validacion valores proximos lo cual no hay indicativo de sobreajuste ni subajuste, el modelo que sobresalio fue Random Forest que si cumplio con f1 score tanto para entrenamiento como validacion.
b) Para sobremuestreo solo los modelos de Random Forest y Decision Tree llegaron y superaron el valor de F1 pero existe la presencia de un sobreajuste por parte del entrenamiento esta aprendiendo las respuestas hay dificultades para generalizar datos no vistos.
b) Compracion entre F1 y AUC-ROC revela fallas notorias entre la capacidad discriminativa del modelo y la efectividad en la identificacion de clientes que abandonan, aunque los modelos  pueden diferenciar entre clases, no se esta logrando un equilibrio adecuado entre precision y recall lo que nos indica un gran numero de falsos negativos